In [33]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from pandas.io.json import json_normalize
import pandas as pd
import numpy as np

import xgboost as xgb
import re 

from tqdm import tqdm

import gensim
import nltk
from nltk.corpus import stopwords
from gensim.models import word2vec as w2v
LabeledSentence = gensim.models.doc2vec.LabeledSentence
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')


In [34]:
import os 
os.getcwd()

'C:\\Users\\thanisb\\Documents\\Competition\\Zindi\\Tech4MentalHealth\\Notebook'

In [42]:
train_DF.to_csv('../data/train_corrected.csv', index = False)
test_DF.to_csv('../data/test_corrected.csv', index = False)

In [ ]:
# Convert the label to OHE
train_DF = pd.concat([train_DF[['ID', 'text']], pd.get_dummies(train_DF.label)], axis = 1)
labels = ['Alcohol', 'Depression', 'Drugs', 'Suicide']
train_DF

In [16]:
train_DF.head()

,ID,text,Alcohol,Depression,Drugs,Suicide
0,SUAVK39Z,i feel that it was better i dream happy,0,1,0,0
1,9JDAGUV3,why do i get hallucinations,0,0,1,0
2,419WR1LQ,i am stressed due to lack of financial support...,0,1,0,0
3,6UY7DX6Q,why is life important,0,0,0,1
4,FYC0FTFB,how could i be helped to go through the depres...,0,1,0,0


In [38]:
def preprocessing(text, remove_stopwords = True):
    try:
        #print(text)
        processed_text = text.lower()
        processed_text = re.sub("[^a-zA-Z]"," ",processed_text)
        processed_text = processed_text.lower().split()
        #print("processed", processed_text)
        words = processed_text
        if remove_stopwords:
                stops = set(stopwords.words("english"))     
                words = [w for w in processed_text if not w in stops]
    except AttributeError:  # handling the case where the token is empty
        words = ''
    
    return words

def review_sentences(review, tokenizer, remove_stopwords=True):
    # 1. Using nltk tokenizer
    try:
        raw_sentences = tokenizer.tokenize(review.strip())
        
    except AttributeError:  # handling the case where the token is empty
        raw_sentences = ''

    if len(raw_sentences) > 1 : raw_sentences = [" ".join(raw_sentences)]

    sentences = []
    # 2. Loop for each sentence
    for raw_sentence in raw_sentences:
        if len(raw_sentence)>0:
            sentences.append(preprocessing(raw_sentence, remove_stopwords))

    # This returns the list of lists
    return sentences



In [39]:
# Training a Word2Vec model
def w2vec_model(text, feature_embed):
    num_features = feature_embed  # Word vector dimensionality
    min_word_count = 1  # Minimum word count
    num_workers = 4     # Number of parallel threads
    context = 10        # Context window size
    downsampling = 1e-3 # (0.001) Downsample setting for frequent words

    print("Training model....")
    model = w2v.Word2Vec(text,
                         workers=num_workers,
                         size=num_features,
                         min_count=min_word_count,
                         window=context,
                         sample=downsampling
    )

    #model.build_vocab(sentence)
    model.train(text, total_examples= model.corpus_count, epochs=300)

    # # To make the model memory efficient
    # model.init_sims(replace=True)

    # # Saving the model for later use. Can be loaded using Word2Vec.load()
    # model_name = "300features_40minwords_10context"
    # model.save(model_name)

    print("Vocabulary shape", model.wv.syn0.shape)
    return model

In [40]:
# Function to average all word vectors in a paragraph
def featureVecMethod(words, model, num_features):
    # Pre-initialising empty numpy array for speed
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0
    
    #Converting Index2Word which is a list to a set for better speed in the execution.
    index2word_set = set(model.wv.index2word)
    
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    
    # Dividing the result by number of words to get average
    featureVec = np.divide(featureVec, nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        # Printing a status message every 1000th review
        #if counter%1000 == 0:
        #    print("Review %d of %d"%(counter,len(reviews)))
            
        reviewFeatureVecs[counter] = featureVecMethod(review, model, num_features)
        counter = counter+1
        
    return reviewFeatureVecs

In [17]:
Feature_dimension = 50

train_content_sentence = []
# TOkenizing all the text to
for i, sent in enumerate(train_DF.text):
    content = review_sentences(sent, tokenizer, remove_stopwords=False)
    train_content_sentence += content
print(len(train_content_sentence))

test_content_sentence = []
for i, sent in enumerate(test_DF.text):
    content = review_sentences(sent, tokenizer, remove_stopwords=False)
    test_content_sentence += content
print(len(test_content_sentence))
    
Overall_content_sentence = train_content_sentence + test_content_sentence
print(len(Overall_content_sentence))


model = w2vec_model(Overall_content_sentence, feature_embed = Feature_dimension)

train_content_embed = getAvgFeatureVecs(train_content_sentence, model, num_features = Feature_dimension)
print(train_content_embed.shape)

test_content_embed = getAvgFeatureVecs(test_content_sentence, model, num_features = Feature_dimension)
print(test_content_embed.shape)

616
309
925
Training model....
Vocabulary shape (999, 50)
(616, 50)
(309, 50)


C:\Users\thanisb\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
C:\Users\thanisb\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [18]:
train_content_embed_DF = pd.DataFrame(train_content_embed)
train_content_embed_DF.columns = ['embed' + "_" + str(x) for x in range(0, Feature_dimension)]
train_content_embed_DF[labels] = train_DF[labels]
train_content_embed_DF['ID'] = train_DF.ID
print("Content Embed shape",train_content_embed_DF.shape)

train_content_embed_DF.head()

Content Embed shape (616, 55)


,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,embed_8,embed_9,...,embed_45,embed_46,embed_47,embed_48,embed_49,Alcohol,Depression,Drugs,Suicide,ID
0,0.230466,-0.182905,-0.332333,0.555513,0.333438,0.052286,0.029042,0.208676,-0.017010,0.382459,...,-0.252820,-0.373233,0.247550,0.719784,0.329593,0,1,0,0,SUAVK39Z
1,0.541881,-0.306691,0.797221,-0.078301,-0.510619,1.026608,0.078903,0.005831,-0.627429,0.446837,...,0.627112,0.852660,0.004821,-0.046888,-0.491767,0,0,1,0,9JDAGUV3
2,-0.102223,0.483680,0.690245,0.056534,-0.078287,-0.295374,-0.022944,0.372534,-0.673362,-1.448769,...,-0.905539,-0.457861,0.053081,-0.063998,0.319653,0,1,0,0,419WR1LQ
3,0.952695,-0.174957,0.483172,-0.312235,0.086527,1.537884,-0.072399,-0.711138,0.101181,0.946507,...,-0.221083,1.092642,0.459013,0.397478,-0.567559,0,0,0,1,6UY7DX6Q
4,-0.294667,-0.573680,0.111463,0.267076,0.362375,0.494306,0.667499,-0.485858,-0.556440,0.203748,...,0.530926,-0.875879,-0.089007,0.131266,0.226533,0,1,0,0,FYC0FTFB


In [19]:
test_content_embed_DF = pd.DataFrame(test_content_embed)
test_content_embed_DF.columns = ['embed' + "_" + str(x) for x in range(0, Feature_dimension)]
test_content_embed_DF['ID'] = test_DF.ID
print("Content Embed shape", test_content_embed_DF.shape)

test_content_embed_DF.head()

Content Embed shape (309, 51)


,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,embed_8,embed_9,...,embed_41,embed_42,embed_43,embed_44,embed_45,embed_46,embed_47,embed_48,embed_49,ID
0,0.503976,-0.529186,0.269241,0.978526,-0.103814,-0.086527,1.040553,0.569527,0.062595,-0.777755,...,-0.300687,0.700590,0.230538,0.467943,-0.170154,-0.039843,0.395929,-0.529277,-0.207355,02V56KMO
1,0.165564,-0.371402,0.301935,-1.030253,0.479653,-0.524703,-0.219242,0.300049,0.401512,-0.024347,...,0.282781,-0.173455,0.058776,0.738854,-0.444026,0.446483,0.787220,1.445460,-0.831845,03BMGTOK
2,-0.138187,-0.631721,0.032339,-0.486305,-0.156812,-0.479171,0.108851,-0.519083,0.283739,-0.401319,...,-0.622994,-0.889408,0.353788,0.863759,-0.636728,-0.163361,0.373822,0.893028,0.078207,03LZVFM6
3,0.014786,-0.946612,0.087856,-0.276736,0.109606,-0.110407,0.312986,-1.168797,0.146922,-0.467349,...,-0.618755,-0.105546,0.909864,0.964325,-0.452088,-0.127764,0.203282,0.636816,0.432266,0EPULUM5
4,-0.117497,-0.052644,-0.101514,-0.095261,0.353081,0.906157,-0.471909,0.086664,0.482570,-0.319598,...,-0.197256,-0.288603,0.221920,-0.763512,0.251746,-0.387666,-0.492219,0.318555,0.408116,0GM4C5GD


In [20]:
train_content_embed_DF.shape, test_content_embed_DF.shape

((616, 55), (309, 51))

In [22]:
dep = labels
indep = train_content_embed_DF.columns.difference(labels + ['ID'])

In [23]:
num_rounds = 10000
params = {'objective' : 'binary:logistic'
          ,'eval_metric': 'logloss'
          ,'max_depth' : 6
          ,'eta' : 0.3
          ,'subsample': 1
          ,'colsample_bytree': 1
          #,'tree_method' : 'gpu_hist'
          }

final_output = pd.DataFrame()
final_output['ID'] = test_DF['ID']
score_dict = {}
for lab in labels:
    print(lab)
    np.random.seed(100)
    train_local_X, valid_local_X, train_local_Y, valid_local_Y = train_test_split(train_content_embed_DF[indep],
                                                                                  train_content_embed_DF[lab],
                                                                                  test_size = 0.2,
                                                                                  random_state = 100,
                                                                                  stratify = train_content_embed_DF[lab])
    
    print(train_local_X.shape, train_local_Y.shape, valid_local_X.shape, valid_local_Y.shape)
    
    dtrain_local = xgb.DMatrix(data = train_local_X, label = train_local_Y)
    dtest_local  = xgb.DMatrix(data = valid_local_X, label = valid_local_Y)
    dtrain_prod = xgb.DMatrix(data = train_content_embed_DF[indep], label = train_content_embed_DF[lab])
    dtest_prod = xgb.DMatrix(data = test_content_embed_DF[indep])

    eval_set = [(dtrain_local,'train'), (dtest_local,'test')]

    np.random.seed(100)
    local_model = xgb.train(params,
                            dtrain_local,
                            evals = eval_set,
                            num_boost_round = num_rounds,
                            verbose_eval = True,
                            early_stopping_rounds = 20
                                       )
    best_iteration = local_model.best_iteration
    best_score = local_model.best_score

    score_dict[lab] = best_score
    
    np.random.seed(100)
    clf = xgb.train(params,
                    dtrain_prod,
                    num_boost_round = best_iteration,
                    evals = eval_set,
                    verbose_eval = True,
                    early_stopping_rounds = 20
                               )

    xgb_prod_predict = clf.predict(dtest_prod)

    final_output[lab] = xgb_prod_predict

final_output = final_output[['ID','Depression', 'Alcohol', 'Suicide', 'Drugs']]

print(score_dict)

Alcohol
(492, 50) (492,) (124, 50) (124,)
[0]	train-logloss:0.49854	test-logloss:0.56275
Multiple eval metrics have been passed: 'test-logloss' will be used for early stopping.

Will train until test-logloss hasn't improved in 20 rounds.
[1]	train-logloss:0.38194	test-logloss:0.49438
[2]	train-logloss:0.30236	test-logloss:0.44479
[3]	train-logloss:0.24049	test-logloss:0.41284
[4]	train-logloss:0.19118	test-logloss:0.38661
[5]	train-logloss:0.16106	test-logloss:0.36276
[6]	train-logloss:0.13380	test-logloss:0.35031
[7]	train-logloss:0.11472	test-logloss:0.35026
[8]	train-logloss:0.09729	test-logloss:0.35422
[9]	train-logloss:0.08458	test-logloss:0.35092
[10]	train-logloss:0.07395	test-logloss:0.33769
[11]	train-logloss:0.06573	test-logloss:0.33459
[12]	train-logloss:0.05880	test-logloss:0.33294
[13]	train-logloss:0.05260	test-logloss:0.33437
[14]	train-logloss:0.04815	test-logloss:0.33813
[15]	train-logloss:0.04350	test-logloss:0.33927
[16]	train-logloss:0.04027	test-logloss:0.33961
[17

[11]	train-logloss:0.05382	test-logloss:0.04182
[12]	train-logloss:0.04751	test-logloss:0.03629
[13]	train-logloss:0.04248	test-logloss:0.03299
[14]	train-logloss:0.03825	test-logloss:0.02917
[15]	train-logloss:0.03423	test-logloss:0.02666
[16]	train-logloss:0.03106	test-logloss:0.02432
[17]	train-logloss:0.02828	test-logloss:0.02253
[18]	train-logloss:0.02588	test-logloss:0.02099
[19]	train-logloss:0.02400	test-logloss:0.01911
[20]	train-logloss:0.02239	test-logloss:0.01834
[21]	train-logloss:0.02096	test-logloss:0.01708
[22]	train-logloss:0.01972	test-logloss:0.01610
[23]	train-logloss:0.01868	test-logloss:0.01524
[24]	train-logloss:0.01775	test-logloss:0.01456
[25]	train-logloss:0.01688	test-logloss:0.01359
Suicide
(492, 50) (492,) (124, 50) (124,)
[0]	train-logloss:0.50949	test-logloss:0.53041
Multiple eval metrics have been passed: 'test-logloss' will be used for early stopping.

Will train until test-logloss hasn't improved in 20 rounds.
[1]	train-logloss:0.40097	test-logloss:0.4

In [ ]:
final_output = final_output[['ID', 'Depression', 'Alcohol', 'Suicide', 'Drugs']]
final_output.to_csv('../output/sub_25_XGBoost.csv', index = False)